In [35]:
import os
from typing import List
from pydantic import BaseModel
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langgraph.graph import StateGraph, END

In [36]:
# Prepare Vectorstore

docs = TextLoader("research_notes.txt",encoding="utf-8").load()
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_documents(docs)
embedding = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(chunks, embedding)
retriever = vectorstore.as_retriever()


In [37]:
import os
from langchain.chat_models import init_chat_model
from dotenv import load_dotenv
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
llm=init_chat_model("openai:gpt-4o")

In [38]:
#  LangGraph State Definition
class RAGCoTState(BaseModel):
        question: str
        sub_steps: List[str] = []
        retrieved_docs: List[Document] = []
        answer: str = ""

In [39]:
#Nodes

def plan_steps(state:RAGCoTState)->RAGCoTState:
        prompt=f"Break the question into 2-3 reasoning steps:  {state.question}"
        result=llm.invoke(prompt).content
        subqs = [line.strip("- ") for line in result.splitlines() if line.strip()]
        return state.model_copy(update={"sub_steps":subqs})

In [40]:
# Retrieve for each step
def retrieve_per_step(state:RAGCoTState)-> RAGCoTState:
        all_docs=[]
        for sub in state.sub_steps:
            docs = retriever.invoke(sub)
            all_docs.extend(docs)
        return state.model_copy(update={"retrieved_docs": all_docs})

In [41]:
# Generate Final Answer
def generate_answer(state: RAGCoTState) -> RAGCoTState:
    context = "".join([doc.page_content for doc in state.retrieved_docs])
    prompt = f"""
    You are answering a complex question using reasoning and retrieved documents.
    Question: {state.question}
    Relevant Information:
    {context}
    Now synthesize a well-reasoned final answer.
    """
    result = llm.invoke(prompt).content.strip()
    return state.model_copy(update={"answer": result})

In [42]:
# LangGraph Graph
builder = StateGraph(RAGCoTState)
builder.add_node("planner", plan_steps)
builder.add_node("retriever", retrieve_per_step)
builder.add_node("responder", generate_answer)
builder.set_entry_point("planner")
builder.add_edge("planner", "retriever")
builder.add_edge("retriever", "responder")
builder.add_edge("responder", END)
graph = builder.compile()


In [43]:
# Run CoT RAG Agent\n",
if __name__ == "__main__":
    query = "what are the additional eperiments in Transformer evaluation?"
    state = RAGCoTState(question=query)
    final = graph.invoke(state)
    print("🪜 Reasoning Steps:", final["sub_steps"])
    print("✅ Final Answer:", final["answer"])

🪜 Reasoning Steps: ['To determine the additional experiments in Transformer evaluation, one could break down the question into the following reasoning steps:', "1. **Identify Evaluation Objectives**: Determine what specific aspects of the Transformer model's performance you want to assess. This might include evaluating generalization ability, robustness to noise, handling of long sequences, interpretability, or computational efficiency.", "2. **Design Experiments for Each Objective**: For each identified objective, design experiments that can effectively measure the model's performance under those criteria. For example:", 'To assess generalization, you might conduct experiments on unseen data or different data distributions.', 'To test robustness, you may introduce syntactic or semantic noise into the input data and measure performance degradation.', 'To evaluate handling of long sequences, you could test the model on tasks with varying sequence lengths.', '3. **Compare Across Models a